In [ ]:
import pandas as pd
import numpy as np
import sklearn as sk

make sure to upload valueset and fhir bundle everytime your runtime shuts down

In [ ]:
df = pd.read_excel('/content/ValueSets (1).xlsx', sheet_name=1)

/usr/local/lib/python3.12/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [ ]:
df.columns = df.iloc[0]
df = df[1:].reset_index(drop=True)

Upload and show RXnorm medication codes for OUR reference

In [ ]:
df_Rx = df[df['Code System'].str.contains('RXNORM') == True]
#df_L[df_L['Code System'] == 'LOINC']
df_Rx

,Name,Code System,Type,Steward,OID,Expansion Status,Expansion Date,Code Count
59,Abemaciclib,RXNORM,Extensional,CancerLinQ,2.16.840.1.113762.1.4.1260.476,Active,2025-09-16,24
87,Abortion Medications,RXNORM,Grouping,NACHC,2.16.840.1.113762.1.4.1221.179,Active,2025-09-16,123
141,Acamprosate,RXNORM,Extensional,NCQA,2.16.840.1.113883.3.464.1003.1154,Not Maintained,2025-09-16,1
153,ACE ARB Prescription,RXNORM,Extensional,Cliniwiz,2.16.840.1.113762.1.4.1200.245,Not Maintained,2025-09-16,284
155,ACE inhibitor allergy RXNORM,RXNORM,Extensional,Optum,2.16.840.1.113762.1.4.1078.45,Active,2025-09-16,279
...,...,...,...,...,...,...,...,...
17040,Zileuton Oral Preparations,RXNORM,Extensional,NHLBI Implementation Science and Health Care I...,2.16.840.1.113762.1.4.1196.468,Not Maintained,2025-09-16,3
17041,Zinc,RXNORM,Extensional,Clinical Architecture,2.16.840.1.113883.3.3616.200.110.102.2091,Deprecated,2023-11-20,65
17042,Ziprasidone,RXNORM,Extensional,NCQA,2.16.840.1.113883.3.464.1003.1141,Not Maintained,2025-09-16,5
17043,Zolmitriptan,RXNORM,Extensional,AAN,2.16.840.1.113762.1.4.1034.656,Active,2025-09-16,4


Upload BioBert for semantic vectors

In [ ]:
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, models

# 2) Build a Sentence-Transformers pipeline around Bio_ClinicalBERT
#    - 'Transformer' loads the HF model
#    - 'Pooling' does mean-pooling over token embeddings
biobert_name = "emilyalsentzer/Bio_ClinicalBERT"
word_model = models.Transformer(biobert_name, max_seq_length=256)
pooling    = models.Pooling(word_model.get_word_embedding_dimension(),
                            pooling_mode_mean_tokens=True)
model = SentenceTransformer(modules=[word_model, pooling])

Import FHIR bundles and turn then into python dicts

In [ ]:
import glob
import json

json_dicts = []  # this will store all JSON files as Python dicts

for path in glob.glob("/content/synthetic data/*.json"): # Corrected path pattern
    with open(path, "r") as f:
        data = json.load(f)   # ← this becomes a dict
        json_dicts.append(data)

# Check results
print(f"Loaded {len(json_dicts)} JSON files.")
if json_dicts: # Added a check to ensure the list is not empty before accessing index 0
    print(type(json_dicts[0]))    # should be <class 'dict'>


Loaded 99 JSON files.
<class 'dict'>


function to turn dates into age

In [ ]:
from datetime import datetime, date

def birthday_to_age(birthdate_str):
    # Expecting format "YYYY-MM-DD"
    birthdate = datetime.strptime(birthdate_str, "%Y-%m-%d").date()
    today = date.today()

    age = today.year - birthdate.year - (
        (today.month, today.day) < (birthdate.month, birthdate.day)
    )
    return age

print(birthday_to_age("1990-04-23"))


35


Reference for extracting patient info

In [ ]:
print(json_dicts[0]['entry'][0]['resource']['birthDate'])

1964-01-12


Creating dataset, keep it anonymous to "remain compliant"

In [ ]:
data = pd.DataFrame(columns=['sex', "age", 'weight'])

Weight lives in the observation resource, medical data is really messy so it could be inside any observation, there are likely tons of them. There might be none, we will pick the latest one.

In [ ]:
from datetime import datetime

# weight is in KGs

def get_latest_weight(bundle):
    """
    Given a FHIR Bundle (as a Python dict), return the most recent
    body weight value (int/float) or None if no weight is found.
    """
    weights = []

    # Collect all weight observations with their dates
    for entry in bundle.get("entry", []):
        resource = entry.get("resource", {})

        if resource.get("resourceType") == "Observation":
            # Check if code matches body weight (LOINC 29463-7 or 3141-9)
            codings = resource.get("code", {}).get("coding", [])
            if not any(c.get("code") in ["29463-7", "3141-9"] for c in codings):
                continue

            # measurement value
            value = resource.get("valueQuantity", {}).get("value")

            # date of measurement
            date = (
                resource.get("effectiveDateTime")
                or resource.get("effectivePeriod", {}).get("start")
            )

            if value is not None and date is not None:
                weights.append((value, date))

    # No weights found
    if not weights:
        return None

    # Helper to parse ISO datetimes like "2021-02-05T00:00:00Z"
    def parse_dt(d):
        return datetime.fromisoformat(d.replace("Z", "+00:00"))

    # Pick the (value, date) pair with the latest date
    latest_value, _latest_date = max(weights, key=lambda x: parse_dt(x[1]))

    # Ensure numeric if possible, but keep int/float "as it came"
    if isinstance(latest_value, (int, float)):
        return latest_value
    try:
        return float(latest_value)
    except (TypeError, ValueError):
        return latest_value  # fallback: return as-is


get_latest_weight(json_dicts[0])

103.6

In [85]:
import re
import pandas as pd

# ---- lists for features ----
sex_list = []
age_list = []
weight_list = []
frequency_list = []
period_list = []
periodUnit_list = []
asNeeded_list = []
code_list = []

text_list = []            # e.g. "Amoxicillin"
dosage_list = []          # e.g. 500.0
unit_list = []            # e.g. "MG"
ingestionType_list = []   # e.g. "Oral Tablet"

for patient_bundle in json_dicts:
    # Patient-level info
    patient_resource = patient_bundle['entry'][0]['resource']
    patient_sex = patient_resource['gender']
    patient_age = birthday_to_age(patient_resource['birthDate'])
    patient_weight = get_latest_weight(patient_bundle)

    for entry in patient_bundle['entry']:
        resource = entry.get('resource', {})
        if resource.get('resourceType') != "MedicationRequest":
            continue

        dosage_instructions = resource.get('dosageInstruction')
        if not dosage_instructions:
            continue

        dosage_instruction = dosage_instructions[0]

        # asNeededBoolean
        current_asNeeded = dosage_instruction.get('asNeededBoolean')

        # timing / repeat block
        timing = dosage_instruction.get('timing', {})
        repeat = timing.get('repeat', {})
        current_frequency = repeat.get('frequency')        # may be None
        current_period = repeat.get('period')              # may be None
        current_periodUnit = repeat.get('periodUnit')      # may be None

        # Medication code + display text
        med_code = None
        med_display = None
        med_cc = resource.get('medicationCodeableConcept')
        if med_cc:
            coding = med_cc.get('coding')
            if coding and len(coding) > 0:
                med_code = coding[0].get('code')
                med_display = coding[0].get('display')

        # require both code and display
        if med_code is None or med_display is None:
            continue

        # Parse display: "Amoxicillin 500 MG Oral Tablet"
        # groups: text, dosage, unit, ingestionType
        m = re.match(r'^(.+?)\s+(\d+(?:\.\d+)?)\s+([^\s]+)\s+(.+)$', med_display)
        if not m:
            # skip weird formats we can't parse
            continue
        base_text, dosage_str, unit, ingestion_type = m.groups()
        dosage_val = float(dosage_str)

        # -----------------------------
        # CASE 1: normal dosing present
        # -----------------------------
        if (current_frequency is not None and
            current_period is not None and
            current_periodUnit is not None):

            sex_list.append(0 if patient_sex == 'female' else 1)
            age_list.append(patient_age)
            weight_list.append(patient_weight)
            frequency_list.append(current_frequency)
            period_list.append(current_period)
            periodUnit_list.append(current_periodUnit)
            asNeeded_list.append(current_asNeeded if current_asNeeded is not None else False)
            code_list.append(med_code)

            text_list.append(base_text)
            dosage_list.append(dosage_val)
            unit_list.append(unit)
            ingestionType_list.append(ingestion_type)

        # -------------------------------------------------
        # CASE 2: asNeeded=True but no frequency/period info
        # → encode frequency/period/periodUnit as -1
        # -------------------------------------------------
        elif current_asNeeded is True:
            sex_list.append(0 if patient_sex == 'female' else 1)
            age_list.append(patient_age)
            weight_list.append(patient_weight)
            frequency_list.append(-1)
            period_list.append(-1)
            periodUnit_list.append("-1")
            asNeeded_list.append(True)
            code_list.append(med_code)

            text_list.append(base_text)
            dosage_list.append(dosage_val)
            unit_list.append(unit)
            ingestionType_list.append(ingestion_type)

        # else: skip entries that don't match either case

# ---- Build base DataFrame ----
data = pd.DataFrame({
    'sex': sex_list,
    'age': age_list,
    'weight': weight_list,
    'frequency': frequency_list,
    'period': period_list,
    'periodUnit_raw': periodUnit_list,
    'asNeeded': asNeeded_list,
    'code': code_list,
    'text': text_list,
    'dosage': dosage_list,
    'unit_raw': unit_list,
    'ingestionType_raw': ingestionType_list,
})

# ---- One-hot encode periodUnit, unit, ingestionType ----
periodUnit_dummies = pd.get_dummies(data['periodUnit_raw'], prefix='periodUnit')
data = pd.concat([data.drop('periodUnit_raw', axis=1), periodUnit_dummies], axis=1)
# Drop the artificial -1 periodUnit column if present
data = data.drop(columns=['periodUnit_-1'], errors='ignore')

unit_dummies = pd.get_dummies(data['unit_raw'], prefix='unit')
ingestion_dummies = pd.get_dummies(data['ingestionType_raw'], prefix='ingestionType')
data = pd.concat(
    [data.drop(['unit_raw', 'ingestionType_raw'], axis=1),
     unit_dummies,
     ingestion_dummies],
    axis=1
)

# ---- BioBERT embeddings for text ----
# assumes `model` is your SentenceTransformer based on Bio_ClinicalBERT
text_embeddings = model.encode(
    data['text'].tolist(),
    batch_size=32,
    show_progress_bar=True
)

# optional: turn embeddings into columns and join to df
import numpy as np
emb_dim = text_embeddings.shape[1]
emb_cols = [f'text_emb_{i}' for i in range(emb_dim)]
emb_df = pd.DataFrame(text_embeddings, columns=emb_cols)

data_with_embeddings = pd.concat([data.reset_index(drop=True), emb_df], axis=1)

# `data` = structured features without embeddings
# `data_with_embeddings` = structured + BioBERT semantic vectors
data_with_embeddings = data_with_embeddings.drop(columns=['text'])

data_with_embeddings


Batches:   0%|          | 0/39 [00:00<?, ?it/s]

,sex,age,weight,frequency,period,asNeeded,code,dosage,periodUnit_d,periodUnit_h,...,text_emb_758,text_emb_759,text_emb_760,text_emb_761,text_emb_762,text_emb_763,text_emb_764,text_emb_765,text_emb_766,text_emb_767
0,1,61,103.6,-1,-1.0,True,849574,220.000,False,False,...,0.246655,0.045754,-0.155270,-0.150752,0.230968,0.263589,0.255226,0.079546,0.246149,0.029201
1,1,61,103.6,4,1.0,False,1049625,325.000,True,False,...,-0.002570,-0.047494,-0.177110,0.161740,0.237101,0.221849,0.435801,0.000780,0.029799,0.183268
2,1,61,103.6,1,3.0,False,245134,0.025,True,False,...,-0.168759,-0.000262,-0.045483,-0.309353,0.644725,0.230800,0.247881,0.348546,-0.124667,-0.005597
3,1,61,103.6,2,1.0,False,1860491,10.000,True,False,...,-0.233050,-0.011515,-0.102053,0.035438,0.345636,0.071771,0.171739,0.263593,0.142561,-0.130123
4,1,61,103.6,4,1.0,False,1049625,325.000,True,False,...,-0.002570,-0.047494,-0.177110,0.161740,0.237101,0.221849,0.435801,0.000780,0.029799,0.183268
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1229,1,54,124.9,-1,-1.0,True,310965,200.000,False,False,...,0.092511,-0.163100,-0.296282,0.061011,0.159354,0.204497,0.062456,0.152177,0.049611,-0.239761
1230,0,6,20.0,-1,-1.0,True,665078,5.000,False,False,...,-0.175157,-0.223163,-0.411241,-0.115960,0.358684,0.338978,-0.117798,0.267493,0.081687,0.059359
1231,0,6,20.0,-1,-1.0,True,1870230,0.300,False,False,...,0.195412,0.019973,-0.389181,-0.275732,-0.064636,0.427605,0.160923,0.499107,0.446516,0.041366
1232,0,6,20.0,3,1.0,False,308192,500.000,True,False,...,0.027176,0.092823,-0.277496,-0.182392,0.126954,-0.040716,-0.087234,0.361667,0.272913,0.140693


When we add new data this might change

In [81]:
data["weight"].isna().sum()

np.int64(0)